In [40]:
from qiskit.aqua.algorithms import NumPyEigensolver#, VQE
from qiskit.algorithms import VQE
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators.symplectic.pauli import Pauli
from qiskit.aqua.operators.legacy import WeightedPauliOperator
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter


from qiskit.utils import QuantumInstance, algorithm_globals

import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, IBMQ, execute
from qiskit import execute, Aer
from qiskit.providers.ibmq import IBMQBackend, least_busy
from qiskit.tools.visualization import circuit_drawer
from qiskit.providers.ibmq import least_busy
from IPython.display import clear_output

import cs_vqe as c
import copy
from qiskit.opflow import X, Z, I
from qiskit.opflow.primitive_ops import PauliOp

In [41]:
ham = {'III':-6.823060333, 'ZII':-0.1110098029, 'IZI':-0.5370907285, 'IIZ':-0.3127149146, 
       'ZZI':0.383637914, 'ZIZ':0.2581256772, 'IZZ':0.2523178271, 'IXX':0.01764480014, 
       'IYY':0.01764480014, 'XXI':0.06593809513, 'YYI':-0.06593809513, 'XIX':0.0121680127, 'YIY':0.0121680127}

terms_noncon = ['III','ZII','ZZI','ZIZ','IXX','IYY','IZI','IZZ','IIZ']

ham_noncon = {q:ham[q] for q in terms_noncon}

In [42]:
ham_q = sum([PauliOp(Pauli(k), ham[k]) for k in ham.keys()])
result = NumPyEigensolver(ham_q).run()
exact_result = float(np.real(result.eigenvalues))
exact_result

-7.952199709435633

In [43]:
model = c.quasi_model(ham_noncon)
model

(['ZII', 'IZZ'],
 ['ZZI', 'IXX'],
 {'III': [[], [], 1],
  'ZII': [['ZII'], [], 1],
  'ZZI': [[], ['ZZI'], 1],
  'ZIZ': [['IZZ'], ['ZZI'], 1],
  'IXX': [[], ['IXX'], 1],
  'IYY': [['IZZ'], ['IXX'], (-1+0j)],
  'IZI': [['ZII'], ['ZZI'], 1],
  'IZZ': [['IZZ'], [], 1],
  'IIZ': [['ZII', 'IZZ'], ['ZZI'], 1]})

In [44]:
fn_form = c.energy_function_form(ham_noncon, model)
fn_form

[2,
 2,
 [[-6.823060333, [], [], 'III'],
  [-0.1110098029, [0], [], 'ZII'],
  [0.383637914, [], [0], 'ZZI'],
  [0.2581256772, [1], [0], 'ZIZ'],
  [0.01764480014, [], [1], 'IXX'],
  [(-0.01764480014+0j), [1], [1], 'IYY'],
  [-0.5370907285, [0], [0], 'IZI'],
  [0.2523178271, [1], [], 'IZZ'],
  [-0.3127149146, [0, 1], [0], 'IIZ']]]

In [45]:
gs_noncon = c.find_gs_noncon(ham_noncon)
gs_noncon

[-7.9513019373,
 [[-1, 1], [-1.0, -5.343446762131902e-11]],
 (['ZII', 'IZZ'],
  ['ZZI', 'IXX'],
  {'III': [[], [], 1],
   'ZII': [['ZII'], [], 1],
   'ZZI': [[], ['ZZI'], 1],
   'ZIZ': [['IZZ'], ['ZZI'], 1],
   'IXX': [[], ['IXX'], 1],
   'IYY': [['IZZ'], ['IXX'], (-1+0j)],
   'IZI': [['ZII'], ['ZZI'], 1],
   'IZZ': [['IZZ'], [], 1],
   'IIZ': [['ZII', 'IZZ'], ['ZZI'], 1]}),
 [2,
  2,
  [[-6.823060333, [], [], 'III'],
   [-0.1110098029, [0], [], 'ZII'],
   [0.383637914, [], [0], 'ZZI'],
   [0.2581256772, [1], [0], 'ZIZ'],
   [0.01764480014, [], [1], 'IXX'],
   [(-0.01764480014+0j), [1], [1], 'IYY'],
   [-0.5370907285, [0], [0], 'IZI'],
   [0.2523178271, [1], [], 'IZZ'],
   [-0.3127149146, [0, 1], [0], 'IIZ']]]]

In [46]:
ep_state = gs_noncon[1]
ep_state

[[-1, 1], [-1.0, -5.343446762131902e-11]]

In [47]:
import cs_vqe as c

order = [2, 1, 0]
reduced_hamiltonians = c.get_reduced_hamiltonians(ham, model, fn_form, ep_state, order)
red_ham = reduced_hamiltonians[3]
#print(red_ham)

red_ham = sum([PauliOp(Pauli(k), red_ham[k]) for k in red_ham.keys()])

3
ham {'III': -6.823060333, 'ZII': -0.1110098029, 'IZI': -0.5370907285, 'IIZ': -0.3127149146, 'ZZI': 0.383637914, 'ZIZ': 0.2581256772, 'IZZ': 0.2523178271, 'IXX': 0.01764480014, 'IYY': 0.01764480014, 'XXI': 0.06593809513, 'YYI': -0.06593809513, 'XIX': 0.0121680127, 'YIY': 0.0121680127} 

rotation: ['pi/2', 'IYI'] 

ham term: III
ham t_set_next: {'III': 1.0} 

ham_next: {'III': -6.823060333} 
 ---------------------------
ham term: ZII
ham t_set_next: {'ZII': 1.0} 

ham_next: {'III': -6.823060333, 'ZII': -0.1110098029} 
 ---------------------------
ham term: IZI
ham t_set_next: {'IXI': -1.0} 

ham_next: {'III': -6.823060333, 'ZII': -0.1110098029, 'IXI': 0.5370907285} 
 ---------------------------
ham term: IIZ
ham t_set_next: {'IIZ': 1.0} 

ham_next: {'III': -6.823060333, 'ZII': -0.1110098029, 'IXI': 0.5370907285, 'IIZ': -0.3127149146} 
 ---------------------------
ham term: ZZI
ham t_set_next: {'ZXI': -1.0} 

ham_next: {'III': -6.823060333, 'ZII': -0.1110098029, 'IXI': 0.5370907285, 'II


ham_next: {'III': -6.823060333, 'ZII': -0.1110098029, 'IZX': -0.5370907285, 'IIX': 0.3127149146, 'ZZX': 0.383637914, 'ZIX': -0.2581256772, 'IZI': -0.2523178271, 'IZZ': 0.01764480014, 'IIZ': 0.01764480014} 
 ---------------------------
ham term: XXZ
ham t_set_next: {'XXX': -1.0} 

ham_next: {'III': -6.823060333, 'ZII': -0.1110098029, 'IZX': -0.5370907285, 'IIX': 0.3127149146, 'ZZX': 0.383637914, 'ZIX': -0.2581256772, 'IZI': -0.2523178271, 'IZZ': 0.01764480014, 'IIZ': 0.01764480014, 'XXX': 0.06593809513} 
 ---------------------------
ham term: YYI
ham t_set_next: {'YYI': 1.0} 

ham_next: {'III': -6.823060333, 'ZII': -0.1110098029, 'IZX': -0.5370907285, 'IIX': 0.3127149146, 'ZZX': 0.383637914, 'ZIX': -0.2581256772, 'IZI': -0.2523178271, 'IZZ': 0.01764480014, 'IIZ': 0.01764480014, 'XXX': 0.06593809513, 'YYI': -0.06593809513} 
 ---------------------------
ham term: XYY
ham t_set_next: {'XYY': 1.0} 

ham_next: {'III': -6.823060333, 'ZII': -0.1110098029, 'IZX': -0.5370907285, 'IIX': 0.312714

In [69]:
alpha = Parameter('α')
beta  = Parameter('β')

anz = QuantumCircuit(3)
"""
anz.sdg(2),anz.h(2),anz.cx(0, 1),anz.cx(1, 2),anz.rz(np.pi/2, 2),anz.cx(1, 2),anz.cx(0, 1),anz.h(2),anz.s(2)

anz.barrier()

anz.sdg(2),anz.h(2),anz.rz(np.pi/2, 2),anz.h(2),anz.s(2)

anz.barrier()

anz.sdg(1),anz.h(1),anz.cx(1, 2),anz.rz(np.pi/2, 2),anz.cx(1, 2),anz.h(1),anz.s(1)

anz.barrier()

anz.sdg(1),anz.h(1),anz.rz(np.pi/2, 1),anz.h(1),anz.s(1)
"""
#end of rotation
anz.barrier()

anz.x(0), anz.x(1), anz.x(2)

anz.barrier()

anz.sdg(1),anz.h(0), anz.h(1),anz.cx(0, 1),anz.rz(2 * alpha, 1),anz.cx(0, 1),anz.h(0), anz.h(1),anz.s(1)

anz.barrier()

anz.sdg(2),anz.h(0), anz.h(2),anz.cx(0, 2),anz.rz(2 * beta,2),anz.cx(0, 2),anz.h(0), anz.h(2),anz.s(2)
#end of ansatz

anz.barrier()

anz.sdg(1),anz.h(1),anz.rz(np.pi/2, 1),anz.h(1),anz.s(1)

anz.barrier()

anz.sdg(1),anz.h(1),anz.cx(1, 2),anz.rz(np.pi/2, 2),anz.cx(1, 2),anz.h(1),anz.s(1)

anz.barrier()

anz.sdg(2),anz.h(2),anz.rz(np.pi/2, 2),anz.h(2),anz.s(2)

anz.barrier()

anz.sdg(2),anz.h(2),anz.cx(0, 1),anz.cx(1, 2),anz.rz(np.pi/2, 2),anz.cx(1, 2),anz.cx(0, 1),anz.h(2),anz.s(2)

print(anz.draw())

      ░ ┌───┐ ░  ┌───┐                           ┌───┐      ░  ┌───┐           »
q_0: ─░─┤ X ├─░──┤ H ├────────■───────────────■──┤ H ├──────░──┤ H ├────────■──»
      ░ ├───┤ ░ ┌┴───┴┐┌───┐┌─┴─┐┌─────────┐┌─┴─┐├───┤┌───┐ ░  └───┘        │  »
q_1: ─░─┤ X ├─░─┤ SDG ├┤ H ├┤ X ├┤ RZ(2*α) ├┤ X ├┤ H ├┤ S ├─░───────────────┼──»
      ░ ├───┤ ░ └─────┘└───┘└───┘└─────────┘└───┘└───┘└───┘ ░ ┌─────┐┌───┐┌─┴─┐»
q_2: ─░─┤ X ├─░─────────────────────────────────────────────░─┤ SDG ├┤ H ├┤ X ├»
      ░ └───┘ ░                                             ░ └─────┘└───┘└───┘»
«                     ┌───┐      ░                                   ░        »
«q_0: ─────────────■──┤ H ├──────░───────────────────────────────────░────────»
«                  │  └───┘      ░ ┌─────┐┌───┐┌─────────┐┌───┐┌───┐ ░ ┌─────┐»
«q_1: ─────────────┼─────────────░─┤ SDG ├┤ H ├┤ RZ(π/2) ├┤ H ├┤ S ├─░─┤ SDG ├»
«     ┌─────────┐┌─┴─┐┌───┐┌───┐ ░ └─────┘└───┘└─────────┘└───┘└───┘ ░ └─────┘»
«q_2: ┤ RZ(2*β) ├┤ X ├┤ H ├┤ S ├─

In [70]:
seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
slsqp = SLSQP(maxiter=1000)

vqe = VQE(anz, optimizer=slsqp, quantum_instance=qi)
vqe_run    = vqe.compute_minimum_eigenvalue(operator=red_ham)
vqe_result = vqe_run.optimal_value# + shift

print(vqe_result, exact_result)

-7.952189926426861 -7.952199709435633


In [30]:
print(red_ham)

-6.823060333 * III
- 0.1110098029 * ZII
- 0.5370907285 * ZIZ
+ 0.3127149146 * ZZZ
+ 0.383637914 * IIZ
- 0.2581256772 * IZZ
- 0.2523178271 * IZI
- 0.01764480014 * ZIX
- 0.01764480014 * ZZX
- 0.06593809513 * YYZ
- 0.06593809513 * YYI
- 0.0121680127 * XYY
+ 0.0121680127 * XXX
